##### 1.SG CTA
Societe Generaleが算出する代表的なCTA指数で、世界の主要な運用会社のパフォーマンスを反映しています。

##### 2.SG Trend
SG CTAから、純粋なトレンドフォローのみを抽出した指数になります。

##### 3.HFRX
Hedge Fund Research（HFR）が算出している指数で、システマティックなCTA戦略を運用するファンドのパフォーマンスを追跡しています。  
当該指数も、トレンドフォロー戦略が中心になります。

##### 4.野村Pimco
野村AMが提供する公募ファンド、PIMCOの提供するTFがベースになります。


<br>

##### USD建 -> JPY建への変換
USD指数のリターンからUSDJPYのヘッジコストを控除する。<br>
例えば、**GBIのローカルベース - 円ヘッジ円ベース = USDJPYのヘッジコスト** と考えられる。


In [1]:
import pandas as pd
import workdays
import datetime
from datetime import datetime, timedelta, date
from xbbg import blp
import pdblp

In [118]:
L = [
    "NEIXCTA Index",
    "NEIXCTAT Index",
    "HFRXSDV Index",
    "0131122C JP Equity",
    "NEIXSTTI Index",
    "NEIXTRND Index",
    "SGIXTFEQ Index",
    "SGIXTFIR Index",
    "SGIXTFFX Index",
    "SGIXTFCY Index",
    "SGIXTFXA Index",
]

BDP = blp.bdp(tickers=L, flds=["long_company_name_realtime","crncy"]).loc[L,:].reset_index()
BDP

,index,long_company_name_realtime,crncy
0,NEIXCTA Index,SG CTA Index,USD
1,NEIXCTAT Index,SG Trend Index,USD
2,HFRXSDV Index,HFRX Macro Systematic Diversified CTA Index,USD
3,0131122C JP Equity,Nomura PIMCO Trend Strategy Fund A Course,JPY
4,NEIXSTTI Index,SG Short-Term Traders Index,USD
5,NEIXTRND Index,SG Trend Indicator,USD
6,SGIXTFEQ Index,SGI Equity Trend Following Index USD - Excess ...,USD
7,SGIXTFIR Index,SGI Interest Rate Trend Following Index,USD
8,SGIXTFFX Index,SGI Forex Trend Following Index USD - Excess R...,USD
9,SGIXTFCY Index,SGI Commodity Trend Following Index,USD


In [120]:
L = [
    "NEIXCTA Index",
    "NEIXCTAT Index",
    "0131122C JP Equity"]

df = blp.bdh(L[:4], "tot_return_index_gross_dvds", "20250331", "20250831", Per="M").reset_index()
df.columns = ["Date","SG_CTA","SG_TF","Nomura"]
df['Date'] = pd.to_datetime(df['Date'])
L = [
    "HFRXSDV Index",
]

df1 = blp.bdh(L[:4], "tot_return_index_gross_dvds", "20250331", "20250831", Per="M")


df.insert(3, "HFR", df1.values)
df.iloc[:,1:] = df.iloc[:,1:].pct_change()
df = df.iloc[1:,:].reset_index(drop=True)


HC = blp.bdh(["SBUS13L Index","SBUS13YC Index"], "px_last", "20250331", "20250831", Per="M").pct_change().iloc[1:,:]
HC = HC["SBUS13YC Index"] - HC["SBUS13L Index"] 
HC.columns = ["HC"]

for i in ["SG_CTA","SG_TF","HFR"]:
    df[i] += HC["HC"].values

df

,Date,SG_CTA,SG_TF,HFR,Nomura
0,2025-04-30,-0.047881,-0.052471,-0.048540,-0.043588
1,2025-05-30,-0.021255,-0.025741,-0.017453,-0.027397
2,2025-06-30,0.006213,0.011789,0.006987,0.013814
3,2025-07-31,-0.001908,-0.004049,-0.004814,0.004007
4,2025-08-29,0.007838,0.022710,0.024682,0.019824
